# imports 

In [19]:
%reset -f

In [20]:
from captum.attr import ShapleyValueSampling
from tqdm import trange

from load_data import load_data
from train_models import *
from segmentation import *
from utils import *
import torch
import os
from tqdm import tqdm
import timeit

# device for torch

In [21]:
from torch.cuda import is_available as is_GPU_available
device = "cuda" if is_GPU_available() else "cpu"

# dictionary mapping predictors to torch vs other, step necessary for Captum 
predictors = {
	'torch' : ['resNet'],
	'scikit' : ['miniRocket','randomForest']
}
segmentation_dict = {"clasp":get_claSP_segmentation, "infogain": get_InformationGain_segmentation, "greedygaussian": get_GreedyGaussian_segmentation, "equal": get_equal_segmentation, "nnsegment": get_NNSegment_segmentation}

# hyper-parameters

In [22]:
# hyperparameters
n_background = 50

# settings
dataset_names = {'MP'}    #{sys.argv[1]}
predictor_names = {'randomForest'}    #{sys.argv[2]} {"randomForest", 'miniRocket', 'resNet'}
segmentation_names = {"clasp","greedygaussian", "equal", "infogain","nnsegment"} #, "nnsegment"}  #, infogain , ,
background_names =  {"average", "zero","sampling"} #{"average", "zero", "sampling"}
normalization_names = {"default", "normalized"}

demo_mode = False
# demo
if demo_mode:
    dataset_names = {'MP'}
    predictor_names = {"randomForest"}
    segmentation_names = {'clasp'}
    background_names = {'zero','sampling'}
    normalization_names = {"default", "normalized"}


# instantiate the dictionary

In [23]:
results = dict.fromkeys(('y_test_true', 'label_mapping', "segments", 'y_test_pred', "attributions"))
for key in results.keys():
    results[key] = dict.fromkeys(dataset_names)
normalization_names = normalization_names | {"default"}


# train model

In [24]:
for dataset_name in dataset_names:
    # init dataset
    # load data
    X_train, X_test, y_train, y_test, enc = load_data(subset='all', dataset_name=dataset_name)
    # for debugging only
    if demo_mode:
        X_test = X_test[:2]
        y_test = y_test[:2]

    n_samples, n_chs, ts_length = X_test.shape

    results['y_test_true'][dataset_name] = y_test
    results['label_mapping'][dataset_name] = enc
    results["attributions"][dataset_name] = dict.fromkeys(segmentation_names)
    results["segments"][dataset_name] = dict.fromkeys(segmentation_names)
    results["y_test_pred"][dataset_name] = dict.fromkeys(segmentation_names)

    predictor_dict = dict()
    # TODO not to save if in demo mode!
    for predictor_name in predictor_names:
        if demo_mode:
            dataset_name=None

        if predictor_name=='resNet':
            clf,preds = train_ResNet(X_train, y_train, X_test, y_test, dataset_name,device=device)
        elif predictor_name=='miniRocket':
            clf,preds = train_miniRocket(X_train, y_train, X_test, y_test, dataset_name)
        elif predictor_name=="randomForest":
            clf, preds = train_randomForest(X_train, y_train, X_test, y_test, dataset_name)
        else:
            raise ValueError("predictor not found")

        predictor_dict[predictor_name] = {"clf": clf, "preds": preds}


training random forest
random forest accuracy is 0.4722689075630252


In [25]:
def initialize_result_dict(X_test,predictor_names,dataset_name,segmentation_name,results):
	init_segments = np.empty((X_test.shape[0], X_test.shape[1]), dtype=object) if X_test.shape[1] > 1 else (
		np.empty(X_test.shape[0], dtype=object))
	results["segments"][dataset_name][segmentation_name] = init_segments.copy()
	results["attributions"][dataset_name][segmentation_name] = dict.fromkeys(predictor_names)
	results["y_test_pred"][dataset_name][segmentation_name] = dict.fromkeys(predictor_names)
	for predictor_name in predictor_names:
		results["attributions"][dataset_name][segmentation_name][predictor_name] = dict.fromkeys(background_names)


def get_sample_info(X_test,y_test,results, id):
	# get current sample and label
	ts, y = X_test[id], torch.tensor(y_test[id:id + 1])
	# get segment and its tensor representation
	current_segments = segmentation_method(ts)[:X_test.shape[1]]
	results['segments'][dataset_name][segmentation_name][i] = current_segments
	mask = get_feature_mask(current_segments, ts.shape[-1])
	mask_list.append(mask)
	ts = torch.tensor(ts).repeat(1, 1, 1)  # TODO use something similar to np.expand_dim?
	ts_list.append(ts)
	y_list.append(y)
	return ts,y,mask


def get_background( background_name, results, normalization_names):

	results["attributions"][dataset_name][segmentation_name][predictor_name][background_name] = dict.fromkeys(
		normalization_names)
	# background data
	if background_name == "zero":
		background_dataset = torch.zeros((1,) + X_train.shape[1:])
	elif background_name == "sampling":
		background_dataset = sample_background(X_train, n_background)
	elif background_name == "average":
		background_dataset = sample_background(X_train, n_background).mean(axis=0, keepdim=True)

	return background_dataset


def get_attribution(ts, mask, background_dataset,y, results ): #    global ts, mask, background_dataset, tmp, y
	# different call depending on predictor type
	if predictor_name in predictors['scikit']:
		# if using random forest flat everything
		if predictor_name == "randomForest":
			ts = ts.reshape(-1, n_chs * ts_length)
			mask = mask.reshape(-1, n_chs * ts_length)
			background_dataset = background_dataset.reshape(-1, n_chs * ts_length)

		tmp = SHAP.attribute(ts, target=y, feature_mask=mask, baselines=background_dataset, additional_forward_args=clf)

	elif predictor_name in predictors['torch']:
		# if use torch make sure everything is on selected device
		ts = ts.to(device);
		y = y.to(device)
		mask = mask.to(device);
		background_dataset = background_dataset.to(device)
		tmp = SHAP.attribute(ts, target=y, feature_mask=mask, baselines=background_dataset)

	# in case of random forest 'un-flatten' result
	if predictor_name=="randomForest":
		tmp = tmp.reshape(-1,X_test.shape[1],X_test.shape[2])

	# lastly store current explanation in the data structure; if sampling store the mean
	results['attributions'][dataset_name][segmentation_name][predictor_name][background_name]["default"][i] = torch.mean(tmp, dim=0).cpu().numpy() if \
		background_name=="sampling" else tmp[0].cpu().numpy()


def get_normalized_results(normalization_names,results):
	if "normalized" in normalization_names:
		weights = np.array(list(
			map(lambda x: list(map(lambda y: lengths_to_weights(change_points_to_lengths(y, X_train.shape[-1])), x)),
			    results["segments"][dataset_name][segmentation_name])))

		results['attributions'][dataset_name][segmentation_name][predictor_name][background_name]["normalized"] = \
			results['attributions'][dataset_name][segmentation_name][predictor_name][background_name]["default"] * weights
	if "default" not in normalization_names:
		del results['attributions'][dataset_name][segmentation_name][predictor_name][background_name]["default"]


In [29]:
starttime = timeit.default_timer()

with torch.no_grad():
    for dataset_name in dataset_names:
        for segmentation_name in segmentation_names:
            initialize_result_dict(X_test,predictor_names,dataset_name,segmentation_name,results)
            segmentation_method = segmentation_dict[segmentation_name]
            
            ts_list = []
            mask_list = []
            y_list = []
            
            for i in range(n_samples) : 
                ts,y,mask = get_sample_info(X_test,y_test, results, i)
            
            for background_name in background_names:
                background_dataset = get_background( background_name, results, normalization_names)
                for predictor_name in predictor_names:
                    # get clf and initialize attributions
                    results['y_test_pred'][dataset_name][segmentation_name][predictor_name]=predictor_dict[predictor_name]["preds"]
                    clf = predictor_dict[predictor_name]["clf"]
                    init_attributions = np.zeros(X_test.shape, dtype=np.float32)
                    for normalization_name in normalization_names:
                        results['attributions'][dataset_name][segmentation_name][predictor_name][background_name][normalization_name] = init_attributions.copy()

                    
                    SHAP = ShapleyValueSampling(clf) if predictor_name in predictors['torch'] else ShapleyValueSampling(forward_classification)
                    
                    with tqdm(total=len(ts_list)) as pbar:
                        for i, (ts, mask, y) in enumerate(zip(ts_list, mask_list, y_list)):
                            # for sampling strategy repeat the ts many times as the background dataset size
                            ts = ts.repeat(background_dataset.shape[0],1,1) if background_name=="sampling" else ts
                            get_attribution (ts, mask, background_dataset,y, results )
                            # update tqdm
                            pbar.update(1)
                            
                    pbar.close()
                    get_normalized_results(normalization_names,results)
                    
print("elapsed time", ( timeit.default_timer() -starttime ) )


0
1


KeyboardInterrupt: 

In [8]:
# dump result to disk
if not demo_mode:
    file_name = "_".join( ("all_results",dataset_name,predictor_name) )
else:
	file_name = "_".join( ("all_results_DEMO_",dataset_name,predictor_name) )
file_path = os.path.join("attributions", file_name)
np.save( file_path, results )

# OLD keep temporarily -  to be deleted

In [8]:
# reference dict!
results = {'y_test_true':
                {"dataset_name":None},
            'label_mapping': 
                {"dataset_name":None},
            "segments":
                {"dataset_name":
                    {"segmentation_name":None}
                },
            'y_test_pred':
                {"dataset_name":
                    {"segmentation_name":
                        {"predictor_name": None}
                    }
                },
            "attributions":
                {"dataset_name":
                    {"segmentation_name":
                        {"predictor_name":
                            {"background_name":
                                {"normalization_name": None}
                            }
                        }
                    }
                }
            }

In [10]:
starttime = timeit.default_timer()

with torch.no_grad():
    for dataset_name in dataset_names:
    
        for segmentation_name in segmentation_names:

            init_segments = np.empty( (X_test.shape[0] , X_test.shape[1]), dtype=object) if X_test.shape[1] > 1  else (np.empty( X_test.shape[0] , dtype=object))
            results["segments"][dataset_name][segmentation_name] = init_segments.copy()
            results["attributions"][dataset_name][segmentation_name] = dict.fromkeys(predictor_names)
            results["y_test_pred"][dataset_name][segmentation_name] = dict.fromkeys(predictor_names)
            for predictor_name in predictor_names:
                results["attributions"][dataset_name][segmentation_name][predictor_name] = dict.fromkeys(background_names)
            segmentation_method = segmentation_dict[segmentation_name]
    
            ts_list = []
            mask_list = []
            y_list = []
    
            for i in range(n_samples) : #
                
                # get current sample and label
                ts, y = X_test[i] , torch.tensor( y_test[i:i+1] )
    
                # get segment and its tensor representation
                current_segments = segmentation_method(ts)[:X_test.shape[1]]
                results['segments'][dataset_name][segmentation_name][i] = current_segments
                mask = get_feature_mask(current_segments,ts.shape[-1])
                mask_list.append(mask)
                ts = torch.tensor(ts).repeat(1,1,1)	#TODO use something similar to np.expand_dim?
                ts_list.append(ts)
                y_list.append(y)
    
            for background_name in background_names:
    
                results["attributions"][dataset_name][segmentation_name][predictor_name][background_name] = dict.fromkeys(normalization_names)
                # background data
                if background_name=="zero":
                    background_dataset = torch.zeros((1,) + X_train.shape[1:])
                elif background_name=="sampling":
                    background_dataset = sample_background(X_train, n_background)
                elif background_name=="average":
                    background_dataset = sample_background(X_train, n_background).mean(axis=0, keepdim=True)
    
                for predictor_name in predictor_names:
    
                    clf = predictor_dict[predictor_name]["clf"]
                    results['y_test_pred'][dataset_name][segmentation_name][predictor_name] = predictor_dict[predictor_name]["preds"]

                    init_attributions = np.zeros(X_test.shape, dtype=np.float32)
                    for normalization_name in normalization_names:
                        results['attributions'][dataset_name][segmentation_name][predictor_name][background_name][normalization_name] = init_attributions.copy()
                    
                    SHAP = ShapleyValueSampling(clf) if predictor_name in predictors['torch'] \
                        else ShapleyValueSampling(forward_classification)

                    #print (background_name, predictor_name , segmentation_name)

                    with tqdm(total=len(ts_list)) as pbar:
                        for i, (ts, mask, y) in enumerate(zip(ts_list, mask_list, y_list)):
    
                            # for sampling strategy repeat the ts many times as the background dataset size
                            ts = ts.repeat(background_dataset.shape[0],1,1) if background_name=="sampling" else ts
                            
                            # different call depending on predictor type
                            if predictor_name in predictors['scikit']:
                                # if using random forest flat everything
                                if predictor_name=="randomForest":
                                    ts = ts.reshape( -1, n_chs*ts_length)
                                    mask = mask.reshape( -1, n_chs*ts_length)
                                    background_dataset = background_dataset.reshape( -1, n_chs*ts_length)
                                
                                tmp = SHAP.attribute( ts, target=y , feature_mask=mask, baselines=background_dataset, additional_forward_args=clf)
                            
                            elif predictor_name in predictors['torch']:
                                # if use torch make sure everything is on selected device
                                ts = ts.to(device); y = y.to(device)
                                mask = mask.to(device) ; background_dataset =  background_dataset.to(device)
                                tmp = SHAP.attribute( ts, target=y , feature_mask=mask, baselines=background_dataset)
                            
                            # 'un-flatten' for randomForest
                            if predictor_name=="randomForest":
                                tmp = tmp.reshape(-1,X_test.shape[1],X_test.shape[2])
    
                            # store current explanation in the data structure; if sampling store the mean
                            results['attributions'][dataset_name][segmentation_name][predictor_name][background_name]["default"][i] = torch.mean(tmp, dim=0).cpu().numpy() if \
                                background_name=="sampling" else tmp[0].cpu().numpy()
                            
                            # update tqdm
                            pbar.update(1)
                    
                    pbar.close()

                    if "normalized" in normalization_names:
                        weights = np.array(list(map(lambda x: list(map(lambda y: lengths_to_weights(change_points_to_lengths(y, X_train.shape[-1])), x)), results["segments"][dataset_name][segmentation_name])))

                        results['attributions'][dataset_name][segmentation_name][predictor_name][background_name]["normalized"] = results['attributions'][dataset_name][segmentation_name][predictor_name][background_name]["default"] * weights
                    if "default" not in normalization_names:
                        del results['attributions'][dataset_name][segmentation_name][predictor_name][background_name]["default"]
                        
print("elapsed time", ( timeit.default_timer() -starttime ) )

100%|██████████| 150/150 [00:35<00:00,  4.17it/s]
/home/davide/miniconda3/envs/segment_shap/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/davide/miniconda3/envs/segment_shap/lib/python3.11/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/home/davide/miniconda3/envs/segment_shap/lib/python3.11/site-packages/numpy/core/_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/davide/miniconda3/envs/segment_shap/lib/python3.11/site-packages/numpy/core/_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/home/davide/miniconda3/envs/segment_shap/lib/python3.11/site-packages/numpy/core/_methods.py:198: RuntimeWarning: invalid value encountered in scalar

elapsed time 575.0744022950003



/home/davide/workspace/PhD/bristolSHAP/utils.py:145: RuntimeWarning: divide by zero encountered in divide
  segment_weights = 1 / lengths
